In [16]:
###apicilient

import requests
import time
import sys
import threading
import json


def get_new_data(patient_id):
    res = requests.get(f"http://tesla.iem.pw.edu.pl:9080/v2/monitor/{patient_id}")
    js = res.json()
    
    return {
        "timestamp":js["trace"]["id"],
        "values": [x["value"] for x in js["trace"]["sensors"]],
        "anomalies":[x["anomaly"] for x in js["trace"]["sensors"]]
        }

In [17]:
###app

stop_collector = False
class DataCollectorThread(threading.Thread):
    def run(self):
        store = get_storage()
       # for i in range(10):
        while True:
            add_measurements("2", get_new_data("2"))
            expire_data(5) 
            time.sleep(1)
            
            if stop_collector:
                print("Stopping on request")
                break

if __name__ == "__main__":
   # print(get_new_data("2"))
   # sys.exit(0)


    init_storage()
    #create_layout()
    
    collector = DataCollectorThread()
    collector.start()
    #print("Waiting...")
    #time.sleep(7)
    
    stop_collector = True
    collector.join()
    print("Finished") 


Stopping on request
Finished


In [13]:
### storage

def init_storage():
    global _storage
    _storage = {}
    return _storage

def get_storage():
    global _storage
    return _storage
    

def add_measurements(patient_id, data):
    st = get_storage()
    if patient_id not in st:
        pd = {
            "patient_id": patient_id,
            "timestamps": [],
            "values": [],
            "anomalies": [],
            "_expires_ts":[]
        }
    else:
        pd = st[patient_id]
        
    pd["timestamps"].append(data["timestamp"])  
    pd["values"].append(data["values"])  
    pd["anomalies"].append(data["anomalies"])  
    pd["_expires_ts"].append(time.time())  
        

def expire_data(secs):
    st = get_storage()
    for pid, pd in st.items():
        ts = time.time()
        while len(pd["_expire_ts"]) > 0 and pd["_expire_ts"][0] < (ts-secs):
            pd["timestamps"].pop(0)
            pd["values"].pop(0)
            pd["anomalies"].pop(0)  
            pd["_expires_ts"].pop(0)

In [18]:
### dash.App
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go 
from dash.dependencies import Input, Output
import numpy as np
from storage import get_storage


def legs_plot():
    #HARDCODE OF THE PATIENT 2
    if "2" in get_storage:
        pd = get_storage()["2"]
        times = np.array(pd["timestamps"])
        values = np.array(pd["values"])
    else:
        times = np.array([0])
        values = np.array([[0]])
    
    print(times)
    print(values)
    
    fig = go.Figure([go.Scatter(x=times, y = values[:,0],\
                    line = dict(color = 'firebrick', width = 4), name = 'L0')
                    ])
    return fig

app = dash.Dash()

def create_layout():
    
    app.layout = html.Div(id='parent', children = [
            html.H1(id = 'H1', children = 'Styling using html components', style = {'textAlign':'center',\
                                                                                   'marginTop':40, 'marginBottom':40}),\
                dcc.Graph(id = 'the_plot', figure = legs_plot()), \
                dcc.Interval(id='interval', interval=1000, n_intervals=0)
                ])
    
    
@app callback(Output('component_id'='the_plot',component_property='figure' ),
                [Input('component_id'='interval',component_property='n_intervals' )])
def graph_update(n_intervals):
    
    print(n_intervals)
    return legs_plot()


SyntaxError: invalid syntax (<ipython-input-18-7cd7a68013d7>, line 41)